In [1]:
# This script will process patch files to remove GP FPs in job folders on OTAR FullTextLoadings
# (c) EMBL-EBI, Jan 2020
#
# Started: 15 Sept 2020
# Updated: 13 Oct  2020

_author_ = 'Santosh Tirunagari'

import glob
import gzip
from bs4 import BeautifulSoup
import lxml
from collections import defaultdict
from tqdm import tqdm
import requests
import random
import sys
import pathlib
import json
import io

In [2]:
import argparse

result_path = '/nfs/production/literature/Santosh_Tirunagari/GP_DS_jsonl_Extracted/ABS/'
pathlib.Path(result_path).mkdir(parents=True, exist_ok=True)



In [3]:
def get_GP_tags(file_soup):
    gene_tags =[]
    for each_ztag in file_soup.find_all('uniprot'):
        gene_tags.append(each_ztag.text)
    return list(set(gene_tags))

In [4]:
def get_DS_tags(file_soup):
    disease_tags = []
    for each_ztag in file_soup.find_all('efo'):
        disease_tags.append(each_ztag.text)
    return list(set(disease_tags))

In [5]:
file_path = '/nfs/production/literature/shyama/Abs20.09/ML_FP/Annot_medline.29697000.xml'

In [25]:
def process_each_file_in_job(each_file_path):
    
    with io.open(each_file_path, 'r', encoding='utf8') as fh:
        abs_text_content = fh.read()
        
    ss = abs_text_content.split('<PubmedArticle xmlns:z="ebistuff">\n')
    subFileBlocks = ss[1:]
    
    article_tags_dict ={}
    
    for each_article in tqdm(subFileBlocks):
        soup = BeautifulSoup(each_article, 'xml')
        
        try:
            pm_id = 'PMID'+soup.find('PMID').text #"article-id"
        except:
            pm_id = 'PMID'
        
        GP_list = get_GP_tags(soup)
        DS_list = get_DS_tags(soup)
#         print(GP_list)
        article_tags_dict['pmid'] = pm_id
        article_tags_dict['GP'] = GP_list
        article_tags_dict['DS'] = DS_list

        with io.open(result_path+'tags_'+each_file_path.split('/')[-1][:-3]+'jsonl', 'a', encoding='utf8') as f:
            json.dump(article_tags_dict,f, ensure_ascii=False)#.decode('unicode-escape').encode('utf8') #,indent = 2
            f.write('\n')

In [26]:
process_each_file_in_job(file_path)

100%|██████████| 568/568 [00:07<00:00, 76.51it/s]


In [18]:
# with open(file_path, 'r') as fh:
#     abs_text_content = fh.read()
    
# ss = abs_text_content.split('<PubmedArticle xmlns:z="ebistuff">\n')
# subFileBlocks = ss[1:]

In [19]:
# soup = BeautifulSoup(subFileBlocks[3], 'xml') #PMID29697001

In [ ]:
# soup

In [20]:
# gtags = soup.find_all('uniprot')
# gtags[0].text

'PPAR-β'

In [21]:
# ss1 = json.dump(gtags[0].text)#.decode('unicode-escape').encode('utf8')
# print(ss1)

TypeError: dump() missing 1 required positional argument: 'fp'